In [1]:
import sklearn

In [2]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
file=r'../data/census_income.csv'

ci=pd.read_csv(file)
ci.drop(['education'],axis=1,inplace=True)
ci['Y']=(ci['Y']==' >50K').astype(int)
cat_cols=ci.select_dtypes(['object']).columns

for col in cat_cols:
    freqs=ci[col].value_counts()
    k=freqs.index[freqs>500][:-1]
    for cat in k:
        name=col+'_'+cat
        ci[name]=(ci[col]==cat).astype(int)
    del ci[col]

x=ci.drop(['Y'],1)
y=ci['Y']
    

C:\Users\Jayanti\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


In [19]:
x.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_ Private,workclass_ Self-emp-not-inc,workclass_ Local-gov,workclass_ ?,...,relationship_ Husband,relationship_ Not-in-family,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ White,race_ Black,sex_ Male,native.country_ United-States,native.country_ Mexico
0,39,77516,13,2174,0,40,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
1,50,83311,13,0,0,13,0,1,0,0,...,1,0,0,0,0,1,0,1,1,0
2,38,215646,9,0,0,40,1,0,0,0,...,0,1,0,0,0,1,0,1,1,0
3,53,234721,7,0,0,40,1,0,0,0,...,1,0,0,0,0,0,1,1,1,0
4,28,338409,13,0,0,40,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [20]:
y

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: Y, Length: 32561, dtype: int32

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [7]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
cross_val_score?

In [10]:
def acc_model(params):
    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, x_train, y_train).mean()
# here you can pass other scoring function and non-default values for cv [ default cv is 5]


In [11]:
param_space = {
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.choice('max_features', range(1,30)),
    'n_estimators': hp.choice('n_estimators', range(100,500)),
    'criterion': hp.choice('criterion', ["gini", "entropy"])}

In [12]:
best = 0

def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

In [13]:
trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=10, trials=trials)
print ('best:')
print (best)

# max_evals =10 is not enough , you should work with value in the range 40-60

new best:                                                                                                              
0.8420225145538376                                                                                                     
{'criterion': 'entropy', 'max_depth': 3, 'max_features': 13, 'n_estimators': 187}                                      
new best:                                                                                                              
0.8420225145538376                                                                                                     
{'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'n_estimators': 397}                                       
new best:                                                                                                              
0.8638282332107172                                                                                                     
{'criterion': 'entropy', 'max_depth': 18

In [14]:
rf=RandomForestClassifier(**{'criterion': 'gini', 'max_depth': 14,
                             'max_features': 20, 'n_estimators': 354})

In [15]:
rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=16, max_features=11, n_estimators=389)

In [16]:
from sklearn.metrics import roc_auc_score

In [17]:
roc_auc_score(y_test,rf.predict_proba(x_test)[:,1])

0.9197218140691353